In [1]:
# load tidy dataset

import pandas as pd
import os.path
import numpy as np
from itertools import combinations
import glob
import sys
import time
from sklearn import svm
from sklearn.model_selection import train_test_split
import pdb

## to increase the cell width of the notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# from sklearn.learning_curve import learning_curve
# from sklearn.grid_search import GridSearchCV
# from sklearn.cross_validation import ShuffleSplit

def loadTidyTimings(tidyTimingsCSV): 
    assert os.path.isfile(tidyTimingsCSV), "desired file does not exist" 
    df = pd.read_csv(tidyTimingsCSV, header = None, names = ["animal", "session", "trial", "stimulus", "onsetFrame", "offsetFrame", "onsetTime", "offsetTime"])
    return df 

def loadTidyCalcium(tidyData): 
    assert os.path.isfile(tidyData), "desired file does not exist" 
    df = pd.read_csv(tidyData, header = None, names = ["date", "animal", "session", "trial", "stimulus", "neuronID", "timePt", "CaSignal"])
    return df 

# helper functions to return the number of distinct types in the provided data frame
getNumNeurons = lambda df: len(np.unique(df['neuronID'].tolist()))
getNeurons = lambda df: np.unique(df['neuronID'].tolist())

getNumTrials = lambda df: len(np.unique(df['trial'].tolist()))
getTrials = lambda df: np.unique(df['trial'].tolist())

getOnsetFrameNum = lambda animal, session, trial: df_timings.loc[(df_timings['animal']==animal) & (df_timings['session']==session) & (df_timings['trial']==trial), ['onsetFrame']].values[0][0]
getOffsetFrameNum = lambda animal, session, trial: df_timings.loc[(df_timings['animal']==animal) & (df_timings['session']==session) & (df_timings['trial']==trial), ['offsetFrame']].values[0][0]


# pass in pre-filtered data set containing data for only one animal and the same session (ie SAME NEURONS!)
def getListsOfTrialIDs(df_animalSession):
    ## get trials for both stimuli
    df_anmlSessStimA = df_animalSession[df_animalSession['stimulus'] == stimA]
    df_anmlSessStimB = df_animalSession[df_animalSession['stimulus'] == stimB]
    print(stimA,stimB)

    ## get lists of trial numbers of each stimuli's presentations 
    trials_stimA = np.unique(df_anmlSessStimA['trial'].tolist())
    trials_stimB = np.unique(df_anmlSessStimB['trial'].tolist())
    print("trial IDs for each stimulus type",trials_stimA,trials_stimB)
    return (trials_stimA,trials_stimB)

## pass in a data frame with only a single animal and session    
def getNumTimePtsPerTrial(df_animalSession, trials_stimA, trials_stimB):

    #### get number of timePts in each trial selected above 
    ## (1 to 3 presentations of the same stimuli exist per session in Prabhat's data)
    numTimePtsPerTrial = np.empty((2,max(len(trials_stimA),len(trials_stimB))))
    numTimePtsPerTrial[:] = np.nan
    stimInd = 0;
    for thisStimTypeTrialNums in [trials_stimA, trials_stimB]:
        trialInd = 0
        for trial in thisStimTypeTrialNums:
            inds_thisTrial = (df_animalSession['trial']==trial)
            tmp_df_thisTrial = df_animalSession[inds_thisTrial] # gives all time points for all neurons
            numNeurons = getNumNeurons(tmp_df_thisTrial) 
            numTimePtsPerTrial[stimInd,trialInd] = np.sum(inds_thisTrial)/numNeurons 
            trialInd += 1
        stimInd += 1
    print('stimuliType x presentationNum (row x col)')    
    print(numTimePtsPerTrial) # rows are for stimuli type; cols are presentation of that stimulus
    return numTimePtsPerTrial

## test candidate comparisons based on whether the number of trials per session and approximate number of timePts match
def areNumTrialsPerStimulusEqual(numTimePtsPerTrial):
    
    ## no trials of either type --> discard this comparison for this animal/session   
    if np.all(np.isnan(numTimePtsPerTrial)):
        print("DISCARDED: neither stimulus type were found for this animal and session")
        return False  # skip to next session (WORK: handle this)
        
    ## different numbers of trials per stimuli/session --> discard this comparison for this animal/session 
    elif np.any(np.isnan(numTimePtsPerTrial)): 
        print("DISCARDED: mismatching numbers of trials per stimulus type for this animal/session")
        return False # skip to next session (WORK: handle this)

    ## FULFILLED here: condition that allows analysis to proceed to attempted data
    elif not np.any(np.isnan(numTimePtsPerTrial)): 
        print("trial numbers match")
    else:
        raise RuntimeError('unexpected trial comparison occurred')
        return False
    
    print("checking approx num of time points")

## input argument generated from getNumTimePtsPerTrial
def areNumTimePtsPerTrialSimilar(numTimePtsPerTrial):
    minTPs, maxTPs, meanTPs, stdTPs = timePtStats(numTimePtsPerTrial)             
    if (np.abs(minTPs-meanTPs) > (threshTPs_stdFromMean * np.abs(meanTPs-stdTPs))) \
        or (np.abs(maxTPs-meanTPs) > (threshTPs_stdFromMean * np.abs(meanTPs-stdTPs))): 
        print("DISCARDED: variance in trial length is above the user's threshold")
        return False # skip to next session (WORK: handle this)

    ### passed all criteria if it made it this far
    return True

## input argument created by getNumTimePtsPerTrial function
def timePtStats(numTimePtsPerTrial):
    minTPs = int(np.amin(numTimePtsPerTrial))
    maxTPs = int(np.amax(numTimePtsPerTrial))
    meanTPs = np.mean(numTimePtsPerTrial)
    stdTPs = np.std(numTimePtsPerTrial)

    ## useful for debugging
#     print('min', minTPs)
#     print('max', maxTPs)
#     print('std', stdTPs)
#     print('mean',meanTPs)
#     print('|min-mean|=',np.abs(minTPs-meanTPs))
#     print('|max-mean|=',np.abs(maxTPs-meanTPs))
#     print('|mean-std|=',np.abs(meanTPs-stdTPs))
#     print('thresh * |mean-std|=',(threshTPs_stdFromMean * np.abs(meanTPs-stdTPs)))
    
    return minTPs, maxTPs, meanTPs, stdTPs

def sameNeuronConcat(df_trunc, truncFrameNum):
    neuronArr_anmlSess_stimA = np.empty((getNumNeurons(df_trunc),truncFrameNum-1)) # -1 for 0 indexing
    neuronArr_anmlSess_stimB = np.empty((getNumNeurons(df_trunc),truncFrameNum-1)) # -1 for 0 indexing
    for stimLst in [trials_stimA, trials_stimB]:
        if np.array_equal(stimLst,trials_stimA) == True:
            print('\nstimulus:', stimA)
        elif np.array_equal(stimLst,trials_stimB) == True:
            print('\nstimulus:', stimB)
        else:
            raise RuntimeError('unexpected trial concatenation condition occurred')
        print('truncation frame: ', truncFrameNum)

        ## create temporary sub matrix of concatenated cells for ONE stimulus
        tmp_neuronsArr_sameStim = np.empty((getNumNeurons(df_trunc),truncFrameNum)) # for 0 indexing
        for trial in stimLst:
            print("appending same neurons in trial: ", trial)

            ##  create temporary sub matrix of same trial all cells
            tmp_neuronsArr_sameStim_sameTrial = np.empty((getNumNeurons(df_trunc),truncFrameNum))
            for neuron in getNeurons(df_trunc):
                tmp_neuronInds = (df_trunc['trial']==trial) & (df_trunc['neuronID']==neuron)
                tmp_neuronSeries = df_trunc.loc[tmp_neuronInds,'CaSignal']
                
                ## pandas to numpy conversion
                tmp_neuronVec = tmp_neuronSeries.as_matrix()
                
                ### WORK
#                 print('neuronVec',np.shape(tmp_neuronVec))
#                 print('neuronsArr_sameStim_sameTrial', np.shape(tmp_neuronsArr_sameStim_sameTrial))
#                 tmp_neuronsArr_sameStim_sameTrial[neuron-1,:] = tmp_neuronVec
                ### 
                
            ## append trials to right of same stim if not the first entry
            if trial == stimLst[0]:
                tmp_neuronsArr_sameStim = np.copy(tmp_neuronsArr_sameStim_sameTrial)
            else:    
                tmp_neuronsArr_sameStim = np.concatenate((tmp_neuronsArr_sameStim, tmp_neuronsArr_sameStim_sameTrial), axis=1)
            print('same stim:', np.shape(tmp_neuronsArr_sameStim))

        ## save concatenated data to output variables
        if np.array_equal(stimLst,trials_stimA):
            neuronArr_anmlSess_stimA = tmp_neuronsArr_sameStim
        elif np.array_equal(stimLst,trials_stimB):
            neuronArr_anmlSess_stimB = tmp_neuronsArr_sameStim
        else:
            raise RuntimeError('unexpected same neuron concatenation state occured')
        
    return neuronArr_anmlSess_stimA, neuronArr_anmlSess_stimB

In [4]:
################ concatenate all .csv files exported from matlab into single pandas dataframe df

# tidy csv file and dir (use makeTidy_Anderson.m to convert Ann's structure to csv)
# tidyDataDir = '/home/orthogonull/a_MHR/aa_research/aa_gitResearch/git_ignored/imagingAnalysis/data/2_tidyCSVformat/'
tidyDataDir = 'C:\\Users\\public.Analysis\\Desktop\\MHR\\git_ignored\data\\2_tidyCSVformat\\'
tidyDataFileTemplate = 'mouse'
tidyDataFileExt = '.csv'
tidyTimingsFileAndPath = 'C:\\Users\\public.Analysis\\Desktop\\MHR\\git_ignored\\data\\2_tidyCSVformat\\stimulusTimings.csv'


print("loading stimulus timings into df_timings")
timingsLst = []
print(tidyTimingsFileAndPath)
timingsLst.append(loadTidyTimings(tidyTimingsFileAndPath))
df_timings = pd.concat(timingsLst)

# get all input files you want to add to the same dataset
dataFiles = np.sort(glob.glob(\
#     "/home/orthogonull/a_MHR/aa_research/aa_gitResearch/git_ignored/imagingAnalysis/data/2_tidyCSVformat/mouse*.csv"))
      "C:\\Users\\public.Analysis\\Desktop\\MHR\\git_ignored\data\\2_tidyCSVformat\\mouse*.csv"))
print("\n data files: \n", dataFiles)

print("\n loading and appending to prior pandas data frame")
dataLst = []

for file in dataFiles:
    print(file)
    dataLst.append(loadTidyCalcium(file))
df = pd.concat(dataLst)

print('finished loading')

############# ALL DATA STORED HERE IN DF
    

loading stimulus timings into df_timings
C:\Users\public.Analysis\Desktop\MHR\git_ignored\data\2_tidyCSVformat\stimulusTimings.csv

 data files: 
 [ 'C:\\Users\\public.Analysis\\Desktop\\MHR\\git_ignored\\data\\2_tidyCSVformat\\mouse1.csv'
 'C:\\Users\\public.Analysis\\Desktop\\MHR\\git_ignored\\data\\2_tidyCSVformat\\mouse2.csv'
 'C:\\Users\\public.Analysis\\Desktop\\MHR\\git_ignored\\data\\2_tidyCSVformat\\mouse3.csv'
 'C:\\Users\\public.Analysis\\Desktop\\MHR\\git_ignored\\data\\2_tidyCSVformat\\mouse4.csv'
 'C:\\Users\\public.Analysis\\Desktop\\MHR\\git_ignored\\data\\2_tidyCSVformat\\mouse5.csv']

 loading and appending to prior pandas data frame
C:\Users\public.Analysis\Desktop\MHR\git_ignored\data\2_tidyCSVformat\mouse1.csv
C:\Users\public.Analysis\Desktop\MHR\git_ignored\data\2_tidyCSVformat\mouse2.csv
C:\Users\public.Analysis\Desktop\MHR\git_ignored\data\2_tidyCSVformat\mouse3.csv
C:\Users\public.Analysis\Desktop\MHR\git_ignored\data\2_tidyCSVformat\mouse4.csv
C:\Users\public.

In [5]:
## survey/search data to prepare for split operation
metaStrs = [['dates','date'],['animals','animal'],['sessions','session'],['maxTrials','trial'],['stimuli','stimulus']]

print("searching over entire data set to get range of various IDs for data (used in subsequent loops)") 

## this dictionary holds useful info regarding the range of inputs to loop/search over subsequently
metaDct = {}
for a,b in metaStrs:
    print(a,b)
    metaDct[a] = np.unique(df[b].tolist())
print(metaDct)


searching over entire data set to get range of various IDs for data (used in subsequent loops)
dates date
animals animal
sessions session
maxTrials trial
stimuli stimulus
{'dates': array(['2017_05_00'],
      dtype='<U10'), 'animals': array([1, 3, 4, 5, 7]), 'sessions': array([1, 2, 3]), 'maxTrials': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21]), 'stimuli': array(['USS', 'baseline', 'female', 'male', 'mineral oil odor',
       'peanut odor', 'pred odor', 'rat', 'tone', 'toy'],
      dtype='<U16')}


In [6]:
######### USER PARAMETERS #########

threshTPs_stdFromMean = 0.75 ## WORK: make this std of each type and not all types

In [16]:
# get all pairs of stimuli
stimCmbTpl = tuple(combinations(metaDct['stimuli'],2)) 


######### MAIN LOOP ##########
df_SVM = pd.DataFrame(columns=('dateOfAnalysis', 'dateOfExperiment', 'animal', 'session', 'stimulusA', 'stimulusB', 'SVM_accuracy'))
ind_comparison = 0;
for (stimA, stimB) in stimCmbTpl:
    print((stimA,stimB))
    
    ## get all data for both trial types
    indsBoth = (df['stimulus']==stimA) | (df['stimulus']==stimB)
    df_bothStimuli = df[indsBoth]

    #### select data by animals and sessions
    for animal in metaDct['animals']:
        print("stimuli comparison num: ", ind_comparison+1)
        print('animal: ', animal)
        for session in metaDct['sessions']:
            print('session:', session)
            
            ## return subselection of data where the same neurons were recorded
            inds_animalSession = (df_bothStimuli['animal'] == animal) & (df_bothStimuli['session'] == session)
            df_animalSession = df_bothStimuli[inds_animalSession]
            try: 
                dateOfExperiment = df_animalSession['date'].values[0]
            except:
                dateOfExperiment = '?'
            print('date of exp:', dateOfExperiment)
            
            # get lists of trial IDs matching stimuli
            trials_stimA, trials_stimB = getListsOfTrialIDs(df_animalSession)
            
            #### skip this comparison <-- if the data don't match in number of trials 
            numTimePtsPerTrial = getNumTimePtsPerTrial(df_animalSession,trials_stimA,trials_stimB)
            if areNumTrialsPerStimulusEqual(numTimePtsPerTrial)==False:
                break     
                
            #### skip this comparison <-- if the data don't match in approx number of timePts
            if areNumTimePtsPerTrialSimilar(numTimePtsPerTrial) == False:
                break
            
            # animal, session 
            
            #### select time points
            minTPs, maxTPs, meanTPs, stdTPs = timePtStats(numTimePtsPerTrial)
            truncLst = []
            timingsMissing = False
            isFirstTrialExamined = True
            minStimulusDuration = []
            for trial in np.concatenate((trials_stimA,trials_stimB)):
                
                ## get stimulus timings
                onsetFrame = getOnsetFrameNum(animal,session,trial)
                offsetFrame = getOffsetFrameNum(animal,session,trial)
                stimulusDuration = offsetFrame - onsetFrame
#                 print('stimulus duration',stimulusDuration)
#                 print('onset frame', onsetFrame)
#                 print('offset frame', offsetFrame)
                
                ## break out of both loops if timings are missing
                if np.isnan(onsetFrame) or np.isnan(offsetFrame):
                    print('stimulus timing missing --> skipping')
                    timingsMissing = True
                    break
                    
                ## set as min if first or has min duration
#                 print('min stim dur', minStimulusDuration)
#                 print('first trial examined', isFirstTrialExamined)
                if isFirstTrialExamined == True:
                    isFirstTrialExamined = False
                    minStimulusDuration = stimulusDuration[()]
                    print('first trial examined --> minStimDur is just the stimDur', minStimulusDuration)
                elif stimulusDuration[()] < minStimulusDuration and isFirstTrialExamined == False:
                    print('new shortest is: ', stimulusDuration)
                    minStimulusDuration = stimulusDuration[()]
                minStimulusDuration = int(minStimulusDuration)
                
            ## timings are missing --> skip the rest of the analysis for this set of data becausing 
            if timingsMissing == True: 
                break   
            
            #### truncate data, then select time points, and save to new df
            truncLst = []
            print('trial concat', np.concatenate((trials_stimA,trials_stimB)))
            for trial in np.concatenate((trials_stimA,trials_stimB)):
                onsetFrame = getOnsetFrameNum(animal,session,trial)
                offsetFrame_chosen = onsetFrame + minStimulusDuration
                print('min stim dur', minStimulusDuration)
                print('onset frame', getOnsetFrameNum(animal,session,trial))
                print('offsetFrame_chosen: ',offsetFrame_chosen)
                
                tmp_inds_trunc = (df_animalSession['trial']==trial) & (df_animalSession['timePt'] >= onsetFrame.astype(int)) & (df_animalSession['timePt'] < offsetFrame_chosen.astype(int))
                tmp_df_trunc = df_animalSession[tmp_inds_trunc]
                truncLst.append(tmp_df_trunc)
            df_trunc = pd.concat(truncLst)
            
            ### just an optional sanity check
            print('num time pts in df_trunc')
            getNumTimePtsPerTrial(df_trunc, trials_stimA, trials_stimB) ### work
            ###
            
            #### concatenate same cells 
            ### loop over and concatenate neurons into the same row if they're the same neuron and stimuli 
            ##      (ie mouse, session, stimuli)
            print(animal,session)
            neuronArr_anmlSess_stimA, neuronArr_anmlSess_stimB = sameNeuronConcat(df_trunc, minStimulusDuration)
            print('shape', np.shape(neuronArr_anmlSess_stimA), np.shape(neuronArr_anmlSess_stimB))
            
            ######### SVM #########
            
            ## create SVM format input by concatenating both classes (stimuli types); y is the labels
            print("stimA, stimB",np.shape(neuronArr_anmlSess_stimA), np.shape(neuronArr_anmlSess_stimB))
            X = np.concatenate((neuronArr_anmlSess_stimA, neuronArr_anmlSess_stimB), axis = 0)
            y = np.empty((neuronArr_anmlSess_stimA.shape[0]+neuronArr_anmlSess_stimB.shape[0]))
            y[:neuronArr_anmlSess_stimA.shape[0]] = 0
            y[neuronArr_anmlSess_stimB.shape[0]:] = 1
            print("X:", X.shape)
            print("y:", y.shape)
            
            print("k fold partitioning")
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
            
            print("training SVM")
            clf = svm.SVC(kernel='linear')
            
            ### WORK
            if ind_comparison == 14:
                np.savetxt("y_train_works.csv", y_train, delimiter=",")
                np.savetxt("X_train_works.csv", X_train, delimiter=",")
            
            try:
                print(ind_comparison)
                clf.fit(X_train, y_train)
            except:
                np.savetxt("y_train_broken.csv", y_train, delimiter=",")
                np.savetxt("X_train_broken.csv", X_train, delimiter=",")
                pdb.set_trace()
            ### WORK
            
            
            print("testing SVM")
            tmp_score = clf.score(X_test, y_test)
            print(tmp_score)
            
            analysisDate = pd.to_datetime('now')
            
            df_SVM.loc[ind_comparison] = [analysisDate, dateOfExperiment, animal, session, stimA, stimB, tmp_score]
            print(df_SVM)

            ind_comparison += 1
            print('\n')
        print('########\n')
print('total number of comparisons: ', ind_comparison+1)

('USS', 'baseline')
stimuli comparison num:  1
animal:  1
session: 1
date of exp: 2017_05_00
USS baseline
trial IDs for each stimulus type [ 5 10 16] [ 1  8 15]
stimuliType x presentationNum (row x col)
[[ 1117.  1116.  1117.]
 [  370.   363.  1476.]]
trial numbers match
checking approx num of time points
DISCARDED: variance in trial length is above the user's threshold
########

stimuli comparison num:  1
animal:  3
session: 1
date of exp: 2017_05_00
USS baseline
trial IDs for each stimulus type [ 4 11 17] [ 1  8 15]
stimuliType x presentationNum (row x col)
[[ 1267.  1127.  1123.]
 [  674.   682.   678.]]
trial numbers match
checking approx num of time points
first trial examined --> minStimDur is just the stimDur 110.0
stimulus timing missing --> skipping
########

stimuli comparison num:  1
animal:  4
session: 1
date of exp: 2017_05_00
USS baseline
trial IDs for each stimulus type [ 4 11] []
stimuliType x presentationNum (row x col)
[[ 1137.  1113.]
 [   nan    nan]]
DISCARDED: mis

stimuliType x presentationNum (row x col)
[[  918.   894.  1068.]
 [  893.   941.  1015.]]
trial numbers match
checking approx num of time points
first trial examined --> minStimDur is just the stimDur 110.0
trial concat [ 6 12 16  3 13 17]
min stim dur 110
onset frame 339.0
offsetFrame_chosen:  449.0
min stim dur 110
onset frame 338.0
offsetFrame_chosen:  448.0
min stim dur 110
onset frame 351.0
offsetFrame_chosen:  461.0
min stim dur 110
onset frame 339.0
offsetFrame_chosen:  449.0
min stim dur 110
onset frame 355.0
offsetFrame_chosen:  465.0
min stim dur 110
onset frame 338.0
offsetFrame_chosen:  448.0
num time pts in df_trunc
stimuliType x presentationNum (row x col)
[[ 110.  110.  110.]
 [ 110.  110.  110.]]
3 2

stimulus: USS
truncation frame:  110
appending same neurons in trial:  6
same stim: (35, 110)
appending same neurons in trial:  12
same stim: (35, 220)
appending same neurons in trial:  16
same stim: (35, 330)

stimulus: female
truncation frame:  110
appending same neuron

stimuliType x presentationNum (row x col)
[[ 1139.  1140.  1232.]
 [ 1134.  1114.  1168.]]
trial numbers match
checking approx num of time points
first trial examined --> minStimDur is just the stimDur 110.0
new shortest is:  104.0
trial concat [ 6 13 16  4 11 19]
min stim dur 104
onset frame 339.0
offsetFrame_chosen:  443.0
min stim dur 104
onset frame 341.0
offsetFrame_chosen:  445.0
min stim dur 104
onset frame 343.0
offsetFrame_chosen:  447.0
min stim dur 104
onset frame 353.0
offsetFrame_chosen:  457.0
min stim dur 104
onset frame 374.0
offsetFrame_chosen:  478.0
min stim dur 104
onset frame 380.0
offsetFrame_chosen:  484.0
num time pts in df_trunc
stimuliType x presentationNum (row x col)
[[ 104.  104.  104.]
 [ 104.  104.  104.]]
1 3

stimulus: USS
truncation frame:  104
appending same neurons in trial:  6
same stim: (21, 104)
appending same neurons in trial:  13
same stim: (21, 208)
appending same neurons in trial:  16
same stim: (21, 312)

stimulus: male
truncation frame:  104

same stim: (34, 110)
appending same neurons in trial:  14
same stim: (34, 220)
appending same neurons in trial:  17
same stim: (34, 330)

stimulus: male
truncation frame:  110
appending same neurons in trial:  7
same stim: (34, 110)
appending same neurons in trial:  9
same stim: (34, 220)
appending same neurons in trial:  18
same stim: (34, 330)
shape (34, 330) (34, 330)
stimA, stimB (34, 330) (34, 330)
X: (68, 330)
y: (68,)
k fold partitioning
training SVM
11
testing SVM
0.785714285714
        dateOfAnalysis dateOfExperiment animal session stimulusA stimulusB  \
0  2017-07-26 05:33:25       2017_05_00      1       1       USS    female   
1  2017-07-26 05:33:25       2017_05_00      1       2       USS    female   
2  2017-07-26 05:33:25       2017_05_00      1       3       USS    female   
3  2017-07-26 05:33:26       2017_05_00      3       1       USS    female   
4  2017-07-26 05:33:26       2017_05_00      3       2       USS    female   
5  2017-07-26 05:33:27       2017_05_00 

trial numbers match
checking approx num of time points
first trial examined --> minStimDur is just the stimDur 111.0
new shortest is:  101.0
trial concat [ 6  9  8 11]
min stim dur 101
onset frame 335.0
offsetFrame_chosen:  436.0
min stim dur 101
onset frame 335.0
offsetFrame_chosen:  436.0
min stim dur 101
onset frame 309.0
offsetFrame_chosen:  410.0
min stim dur 101
onset frame 342.0
offsetFrame_chosen:  443.0
num time pts in df_trunc
stimuliType x presentationNum (row x col)
[[ 101.  101.]
 [ 101.  101.]]
5 1

stimulus: USS
truncation frame:  101
appending same neurons in trial:  6
same stim: (36, 101)
appending same neurons in trial:  9
same stim: (36, 202)

stimulus: male
truncation frame:  101
appending same neurons in trial:  8
same stim: (36, 101)
appending same neurons in trial:  11
same stim: (36, 202)
shape (36, 202) (36, 202)
stimA, stimB (36, 202) (36, 202)
X: (72, 202)
y: (72,)
k fold partitioning
training SVM
15
testing SVM
0.866666666667
        dateOfAnalysis dateOfExp

...

stimulus: toy
truncation frame:  111
appending same neurons in trial:  8
same stim: (56, 111)
appending same neurons in trial:  10
same stim: (56, 222)
shape (56, 222) (56, 222)
stimA, stimB (56, 222) (56, 222)
X: (112, 222)
y: (112,)
k fold partitioning
training SVM
303
testing SVM
0.478260869565
         dateOfAnalysis dateOfExperiment animal session stimulusA  \
0   2017-07-26 05:33:25       2017_05_00      1       1       USS   
1   2017-07-26 05:33:25       2017_05_00      1       2       USS   
2   2017-07-26 05:33:25       2017_05_00      1       3       USS   
3   2017-07-26 05:33:26       2017_05_00      3       1       USS   
4   2017-07-26 05:33:26       2017_05_00      3       2       USS   
5   2017-07-26 05:33:27       2017_05_00      3       3       USS   
6   2017-07-26 05:33:29       2017_05_00      1       1       USS   
7   2017-07-26 05:33:30       2017_05_00      1       2       USS   
8   2017-07-26 05:33:30       2017_05_00      1       3       USS   
9   2017-07

date of exp: 2017_05_00
tone toy
trial IDs for each stimulus type [ 2 15] [7 9]
stimuliType x presentationNum (row x col)
[[ 1119.  1123.]
 [ 1120.  1152.]]
trial numbers match
checking approx num of time points
first trial examined --> minStimDur is just the stimDur 111.0
trial concat [ 2 15  7  9]
min stim dur 111
onset frame 336.0
offsetFrame_chosen:  447.0
min stim dur 111
onset frame 342.0
offsetFrame_chosen:  453.0
min stim dur 111
onset frame 334.0
offsetFrame_chosen:  445.0
min stim dur 111
onset frame 329.0
offsetFrame_chosen:  440.0
num time pts in df_trunc
stimuliType x presentationNum (row x col)
[[ 111.  111.]
 [ 111.  111.]]
4 3

stimulus: tone
truncation frame:  111
appending same neurons in trial:  2
same stim: (47, 111)
appending same neurons in trial:  15
same stim: (47, 222)

stimulus: toy
truncation frame:  111
appending same neurons in trial:  7
same stim: (47, 111)
appending same neurons in trial:  9
same stim: (47, 222)
shape (47, 222) (47, 222)
stimA, stimB (47,

session: 2
date of exp: 2017_05_00
tone toy
trial IDs for each stimulus type [8] [4]
stimuliType x presentationNum (row x col)
[[ 1119.]
 [ 1113.]]
trial numbers match
checking approx num of time points
first trial examined --> minStimDur is just the stimDur 111.0
trial concat [8 4]
min stim dur 111
onset frame 338.0
offsetFrame_chosen:  449.0
min stim dur 111
onset frame 309.0
offsetFrame_chosen:  420.0
num time pts in df_trunc
stimuliType x presentationNum (row x col)
[[ 111.]
 [ 111.]]
5 2

stimulus: tone
truncation frame:  111
appending same neurons in trial:  8
same stim: (35, 111)

stimulus: toy
truncation frame:  111
appending same neurons in trial:  4
same stim: (35, 111)
shape (35, 111) (35, 111)
stimA, stimB (35, 111) (35, 111)
X: (70, 111)
y: (70,)
k fold partitioning
training SVM
307
testing SVM
0.428571428571
         dateOfAnalysis dateOfExperiment animal session stimulusA  \
0   2017-07-26 05:33:25       2017_05_00      1       1       USS   
1   2017-07-26 05:33:25     

tone toy
trial IDs for each stimulus type [ 6 11] [ 1 12]
stimuliType x presentationNum (row x col)
[[ 1009.  1016.]
 [ 1016.  1015.]]
trial numbers match
checking approx num of time points
first trial examined --> minStimDur is just the stimDur 111.0
trial concat [ 6 11  1 12]
min stim dur 111
onset frame 298.0
offsetFrame_chosen:  409.0
min stim dur 111
onset frame 320.0
offsetFrame_chosen:  431.0
min stim dur 111
onset frame 287.0
offsetFrame_chosen:  398.0
min stim dur 111
onset frame 292.0
offsetFrame_chosen:  403.0
num time pts in df_trunc
stimuliType x presentationNum (row x col)
[[ 111.  111.]
 [ 111.  111.]]
7 1

stimulus: tone
truncation frame:  111
appending same neurons in trial:  6
same stim: (57, 111)
appending same neurons in trial:  11
same stim: (57, 222)

stimulus: toy
truncation frame:  111
appending same neurons in trial:  1
same stim: (57, 111)
appending same neurons in trial:  12
same stim: (57, 222)
shape (57, 222) (57, 222)
stimA, stimB (57, 222) (57, 222)
X: (1

stimA, stimB (40, 222) (40, 222)
X: (80, 222)
y: (80,)
k fold partitioning
training SVM
310
testing SVM
0.25
         dateOfAnalysis dateOfExperiment animal session stimulusA  \
0   2017-07-26 05:33:25       2017_05_00      1       1       USS   
1   2017-07-26 05:33:25       2017_05_00      1       2       USS   
2   2017-07-26 05:33:25       2017_05_00      1       3       USS   
3   2017-07-26 05:33:26       2017_05_00      3       1       USS   
4   2017-07-26 05:33:26       2017_05_00      3       2       USS   
5   2017-07-26 05:33:27       2017_05_00      3       3       USS   
6   2017-07-26 05:33:29       2017_05_00      1       1       USS   
7   2017-07-26 05:33:30       2017_05_00      1       2       USS   
8   2017-07-26 05:33:30       2017_05_00      1       3       USS   
9   2017-07-26 05:33:30       2017_05_00      3       1       USS   
10  2017-07-26 05:33:31       2017_05_00      3       2       USS   
11  2017-07-26 05:33:31       2017_05_00      3       3       U

In [ ]:
## buffer cell
pdb.set_trace()

np.savetxt("X_train.csv", X_train, delimiter=",")

            #             tmp_SVMresult = pd.DataFrame({"animal": [animal]})#,{"testAccuracy": tmp_score})
#             tmp_SVMresult = pd.DataFrame([animal], columns = list([1])) #,{"testAccuracy": tmp_score})
#             df_SVM.loc[df_SVM.index.max() + 1] = [animal, session, tmp_score]
#             df_SVM.loc[totalNumComparisons] = [animal, session, tmp_score]
#             df_SVM.loc[totalNumComparisons] = [0,1,2]
#             df_SVM = pd.DataFrame({'mouse': animal, 'session': session, 'SVMaccuracy': tmp_score}, index =totalNumComparisons)
#             df_SVM.iloc[1] = dict(x=9, y=99)
#             df_SVM.append(tmp_SVMresult, ignore_index=True)
#             clf.predict(X_test, y_test)
            
    
    ##########33
#             cv = ShuffleSplit(X_train.shape[0], n_iter=10, test_size=0.2, random_state=0)
            
            # WORK: optional gridsearch
#             gammas = np.logspace(-6, -1, 10)
#             classifier = GridSearchCV(estimator=estimator, cv=cv, param_grid=dict(gamma=gammas))
#             classifier.fit(X_train, y_train)
            
#             title = 'Learning Curves (SVM, linear kernel, $\gamma=%.6f$)' %classifier.best_estimator_.gamma
#             estimator = SVC(kernel='linear', gamma=classifier.best_estimator_.gamma)
#             plot_learning_curve(estimator, title, X_train, y_train, cv=cv)
#             plt.show()




In [ ]:
## export timings
df_timings.to_csv('df_timings.csv', header=True, index=True)

In [17]:
## save raw svm (animal/session) results to /gitTracked/python/
timestr = time.strftime("%Y_%m_%d")
print(timestr)

# Write out raw analysis to csv file
OUTPUT_FILENAME = 'SVM_analysis_raw' + timestr + '.csv'
df_SVM.to_csv(OUTPUT_FILENAME, header=True, index=False)

2017_07_25


In [18]:
## save summary svm data

# svmGrouped, df_SVM_summary = df_SVM.groupby(['stimulusA', 'stimulusB'], as_index=False)

# f = {'SVM_accuracy':['sum','mean'], 'B':['prod']}

stimGrouped = df_SVM.groupby(['stimulusA','stimulusB'], as_index=True)
df_SVM_summaryDesc = stimGrouped.describe()

print(stimGrouped.describe())      

# Write out raw analysis to csv file
OUTPUT_FILENAME = 'SVM_analysis_summary' + timestr + '.csv'
df_SVM_summaryDesc.to_csv(OUTPUT_FILENAME, header=True, index=True)

                                  SVM_accuracy                                \
                                         count      mean       std       min   
stimulusA        stimulusB                                                     
USS              female                    6.0  0.482540  0.277936  0.142857   
                 male                     15.0  0.548004  0.236148  0.142857   
                 mineral oil odor          9.0  0.492307  0.225590  0.045455   
                 peanut odor               7.0  0.528649  0.288981  0.100000   
                 pred odor                 7.0  0.435541  0.085411  0.250000   
                 rat                      13.0  0.501729  0.231614  0.100000   
                 tone                     15.0  0.510403  0.278044  0.000000   
                 toy                      15.0  0.435306  0.222836  0.045455   
female           male                      6.0  0.595899  0.291263  0.142857   
                 rat                    

In [ ]:
## inspect grouped-by object created above

for key, item in stimGrouped:
    print(stimGrouped.get_group(key), "\n\n")

In [ ]:
ratData = df.loc[df['stimulusType'] == 'rat',:]
ussData = df.loc[df['stimulusType'] == 'USS',:]

print(ussData)



df.corr()


# Rename the impact force column
df = df.rename(columns={'impact force (mN)': 'impf'})

